# Images Processing
Here three images datasets are processed in order to select the images to be used as input during experiment. A total of three datasets containing screenshots are analyzed. The screenshots were taken after reconstructing the participant's walk from a free exploration walk in a virtual city. Both head and eye tracking data were used and the images where taken using the eye tracking data. Images were saved to a "screenshots" folder with a respective csv file containing the information for each image that was taken.

**TODO:**
- _Is there  anything interesting in the EYE-Local data?_

## 1. Preprocessing
When looking into the `df_b81` file, we found its dimensions are 163661 x 24, and it contains missing values which should be fixed or deleted.

In [120]:
import numpy as np
import pandas as pd
import os
import shutil
import glob

### 1.1 Join reports
Due to memory constrains, two out of the three replays creating the screenshots and the respective report had to be done in two sessions. In the first session, screenshots and report are automatically done from the participant's "during-walk-gazes" until the unity stops and crashes. Because we know the frame where the last picture is taken, in the second session, the replay is done and only after the frame is bigger than the frame from the latest picture taken, unity continues taking screenshots and making the respective report. Thus, we need to join report number one and number two for those reports with IDs that have two csv files.

In [121]:
r_path = 'data/reports'
file1 = []
file2 = []
# access files in reports  folder and split
for file in os.listdir(r_path):
    if file.lower().endswith('.csv'):
        fileparts = file.split('.')
        # select file 1 and 2 separately
        if fileparts[0].endswith('_1'):
            file1.append(fileparts[0])
        if fileparts[0].endswith('_2'):
            file2.append(fileparts[0])

In [122]:
# if saving to separate folder
path_joined = r_path + '/joined'
for f1 in file1:
    for f2 in file2:
        # compare files ids and concatenate them when they match
        if f1[:-2] == f2[:-2]:
            if f1.endswith('_1'):
                df = pd.concat(map(pd.read_csv, [r_path + '/' + f1 + '.csv', r_path + '/' + f2 + '.csv']), ignore_index=True)
                # in case you want to save to separate folder
                if not os.path.isdir(path_joined):
                    os.mkdir(path_joined)
                # save joined csv files
                df.to_csv(os.path.join(path_joined,f1[:-2]+'_joined'+'.csv'), index=False)

In [123]:
df_81 = pd.read_csv(path_joined  + '/' + file1[0][:-2]+'_joined'+'.csv')
df_81

,dateTime,hitName,rotation,distance,frame,%replayed,toUse
0,22-07-07_19-12-53,baum ld0(enhanced),180,21,1,0,0
1,22-07-07_19-12-54,baum ld0(enhanced),19,52,3,0,0
2,22-07-07_19-12-54,baum ld0(enhanced),19,52,15,0,0
3,22-07-07_19-12-55,baum ld0(enhanced),19,52,19,0,0
4,22-07-07_19-12-55,baum ld0(enhanced),20,52,25,0,0
...,...,...,...,...,...,...,...
8347,22-07-07_21-44-41,Cathedral,33,45,124960,76,0
8348,22-07-07_21-44-41,Cathedral,33,45,124965,76,0
8349,22-07-07_21-44-42,Cathedral,33,45,124970,76,0
8350,22-07-07_21-44-42,Cathedral,33,45,124975,76,0


### 1.2 Moving selected images

In [125]:
j_path = 'data/reports/joined'

report_id = []

for file in os.listdir(j_path):
    if file.lower().endswith('.csv'):
        report_parts = file.split('.')
        report_parts = report_parts[0].split('_')
        r_id = report_parts[2] + '_' + report_parts[3] + '_' +report_parts[4]
        report_id.append(r_id)
report_id

['BodyGazes_correTS_98ea619d-e75e-49b1-986d-07c3bf90ff6c',
 'gazes_correTS_02715e7a-5a68-471b-9e63-f056c5f0df81',
 'BodyGazes_correTS_1dea5bd2-80d1-4068-b512-99c55f85f44c',
 'gazes_correTS_1dea5bd2-80d1-4068-b512-99c55f85f44c',
 'BodyGazes_correTS_02715e7a-5a68-471b-9e63-f056c5f0df81',
 'gazes_correTS_98ea619d-e75e-49b1-986d-07c3bf90ff6c']

In [129]:
img_path = '/Volumes/Madrid/Screenshots'

imgs_id = []

for file in os.listdir(img_path):
    if file.lower():
        img_parts = file.split('_')
        # print(img_parts)
        img_id = img_parts[2] + '_' + img_parts[3] + '_' +img_parts[4]
        imgs_id.append(img_id)

In [133]:
# folder to move the images
output_folder = 'data/images'

for report in report_id:
    for file in os.listdir(j_path):
        if report in file:
            report_file = os.path.join(j_path, file)
            break

    for path in os.listdir(img_path):
        if report in path:
            image_path = os.path.join(img_path, path)
            image_path_name = path
            break

    report_dataset = pd.read_csv(report_file)
    report_dataset = report_dataset[report_dataset['toUse'] == 1]

    print(f"Working on {image_path} ...")

    for _, row in report_dataset.iterrows():
        image_name = 'img.1600x1000.date.20' + row['dateTime'][:-4] + '*-**.hitname.' + row['hitName'] + '.rotation.' + str(row['rotation'])  + '.distance.' + str(row['distance'])  + '.frame.' + str(row['frame']) + '.png'
        files = glob.glob(os.path.join(image_path, image_name))
        if len(files) == 0:
            print(f"File Not Found: {image_name}")
        else:
            # copy the images in the respective folder from the external memory without overwriting them
            if 'face' in row['hitName'].lower():
                if not os.path.isfile(os.path.join(output_folder + '/faces', os.path.basename(files[0]))):
                    print(f"Copying FACE {os.path.basename(files[0])}...")
                    shutil.copy(files[0], os.path.join(output_folder + '/faces', os.path.basename(files[0])))
            elif 'npc' in row['hitName'].lower():
                if not os.path.isfile(os.path.join(output_folder + '/bodies', os.path.basename(files[0]))):
                    print(f"Copying BODY {os.path.basename(files[0])}...")
                    shutil.copy(files[0], os.path.join(output_folder + '/bodies', os.path.basename(files[0])))
            else:
                if not os.path.isfile(os.path.join(output_folder + '/objects', os.path.basename(files[0]))):
                    print(f"Copying objects {os.path.basename(files[0])}...")
                    shutil.copy(files[0], os.path.join(output_folder + '/objects', os.path.basename(files[0])))

Working on /Volumes/Madrid/Screenshots/screenshots_new_BodyGazes_correTS_98ea619d-e75e-49b1-986d-07c3bf90ff6c ...
Working on /Volumes/Madrid/Screenshots/screenshots_new_gazes_correTS_02715e7a-5a68-471b-9e63-f056c5f0df81 ...
Copying FACE img.1600x1000.date.2022-07-07_19-16-01.hitname.face_422.rotation.47.distance.57.frame.6921.png...
Copying FACE img.1600x1000.date.2022-07-07_19-16-23.hitname.face_428.rotation.98.distance.30.frame.7646.png...
File Not Found: img.1600x1000.date.2022-07-07_19-2*-**.hitname.face_619.rotation.16.distance.95.frame.23636.png
Copying FACE img.1600x1000.date.2022-07-07_19-33-21.hitname.face_603.rotation.25.distance.12.frame.39520.png...
File Not Found: img.1600x1000.date.2022-07-07_19-3*-**.hitname.face_603.rotation.93.distance.5.frame.42337.png
Copying FACE img.1600x1000.date.2022-07-07_19-36-57.hitname.face_4203.rotation.15.distance.2.frame.45894.png...
Copying FACE img.1600x1000.date.2022-07-07_19-37-00.hitname.face_4208.rotation.54.distance.1.frame.46004.pn